# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Finding Donors for Charity-ML

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

## Getting Started

In this project, you will employ several supervised algorithms of your choice to accurately model individuals' income using data collected from the 1994 U.S. Census. You will then choose the best candidate algorithm from preliminary results and further optimize this algorithm to best model the data. Your goal with this implementation is to construct a model that accurately predicts whether an individual makes more or less than $50,000.

The dataset for this project originates from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Census+Income). The datset was donated by Ron Kohavi and Barry Becker, after being published in the article _"Scaling Up the Accuracy of Naive-Bayes Classifiers: A Decision-Tree Hybrid"_. You can find the article by Ron Kohavi [online](https://www.aaai.org/Papers/KDD/1996/KDD96-033.pdf). The data we investigate here consists of small changes to the original dataset, such as removing the `'fnlwgt'` feature and records with missing or ill-formatted entries.

----
## Exploring the Data
WIP DISCUSSION

Run the code cell below to load necessary Python libraries and load the census data. Note that the last column from this dataset, `'income'`, will be our target label (whether an individual makes more than, or at most, $50,000 annually). All other columns are features about each individual in the census database.

In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import visuals as vs
from IPython.display import display # Allows the use of display() for DataFrames

# Pretty display for notebooks
%matplotlib inline

# Load the Census dataset
data = pd.read_csv("census.csv")
income = data['income']
features = data.drop('income', axis = 1)

# Success - Display the first record
display(data.head(n=1))

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many individuals fit into each group, and learn about the percentage of these individuals making more than \$50,000. In the code cell below, you will need to compute the following:
- The total number of records, `'n_records'`
- The number of individuals making more than \$50,000 annually, `'n_greater_50k'`.
- The number of individuals making at most \$50,000 annually, `'n_at_most_50k'`.
- The percentage of individuals making more than \$50,000 annually, `'greater_percent'`.

**Hint:** You may need to look at the table above to understand how the `'income'` entries are formatted. 

In [ ]:
# TODO: Total number of records
n_records = None

# TODO: Number of records where individual's income is more than $50,000
n_greater_50k = None

# TODO: Number of records where individual's income is at most $50,000
n_at_most_50k = None

# TODO: Percentage of individuals whose income is more than $50,000
greater_percent = None

# Print the results
print "Total number of records: {}".format(n_records)
print "Individuals making more than $50,000: {}".format(n_greater_50k)
print "Individuals making at most $50,000: {}".format(n_at_most_50k)
print "Percentage of individuals making more than $50,000: {:.2f}%".format(greater_percent)

----
## Preparing the Data
WIP DISCUSSION

### Transforming Skewed Continuous Features
A dataset will often contain at least one feature whose values tend to lie near a single number, but that feature will also have a non-trivial number of vastly larger or smaller values than that single number.  Algorithms can be sensitive to such distributions of values and can underperform if the range is not properly normalized. With the census dataset two features fit this description: '`capital-gain'` and `'capital-loss'`. 

Run the code cell below to plot a histogram of these two features. Note the range of the values present and how they are distributed.

In [ ]:
# Visualize skewed continuous features of original data
vs.distribution(data)

For highly-skewed feature distributions such as `'capital-gain'` and `'capital-loss'`, it is common practice to apply a <a href="https://en.wikipedia.org/wiki/Data_transformation_(statistics)">logarithmic transformation</a> on the data so that the very large and very small values do not negatively affect the performance of a learning algorithm. Using a logarithmic transformation significantly reduces the range of values caused by outliers. Care must be taken when applying this transformation however: The logarithm of `0` is undefined, so we must translate the values by a small amount above `0` to apply the the logarithm successfully.

Run the code cell below to perform a transformation on the data and visualize the results. Again, note the range of values and how they are distributed. 

In [ ]:
# Log-transform the skewed features
skewed = ['capital-gain', 'capital-loss']
features[skewed] = data[skewed].apply(lambda x: np.log(x + 1))

# Visualize the new log distributions
vs.distribution(features, transformed = True)

### Standardizing Numerical Features
In addition to performing transformations on features that are highly skewed, it is often good practice to standardize all numerical features such that the data has a mean of zero and a standard deviation of one. Applying a standardization to the data does not change the shape of each feature's distribution (such as `'capital-gain'` or `'capital-loss'` above; however, standardization ensures that each feature is treated equally when applying supervised learners. A common example of where standardization is beneficial is for learners that use distance between datapoints as a learning metric.

Run the code cell below to standardize each numerical feature. We will use [`'sklearn.preprocessing.StandardScaler'`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) for this.

In [ ]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import StandardScaler

# Initialize a scaler, then apply it to the features
scaler = StandardScaler()
num_features = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
features[num_features] = scaler.fit_transform(data[num_features])

# Show an example of a record with scaling applied
display(features.head(n=1))

### Implementation: Data Preprocessing

From the table in **Exploring the Data** above, we can see there are several features for each record that are non-numeric. Typically, learning algorithms expect input to be numeric, which requires that non-numeric features (called *categorical variables*) be converted. One popular way to convert categorical variables is by using the **one-hot encoding** scheme. One-hot encoding creates a _"dummy"_ variable for each possible category of each non-numeric feature. For example, assume `someFeature` has three possible entries: `A`, `B`, or `C`. We then encode this feature into `someFeature_A`, `someFeature_B` and `someFeature_C`.

|   | someFeature |                    | someFeature_A | someFeature_B | someFeature_C |
| :-: | :-: |                            | :-: | :-: | :-: |
| 0 |  B  |  | 0 | 1 | 0 |
| 1 |  C  | ----> one-hot encode ----> | 0 | 0 | 1 |
| 2 |  A  |  | 1 | 0 | 0 |

Additionally, as with the non-numeric features, we need to convert the non-numeric target label, `'income'` to numerical values for the learning algorithm to work. Since there are only two possible categories for this label ("<=50K" and ">50K"), we can avoid using one-hot encoding and simply encode these two categories as `-1` and `0`, respectively. In code cell below, you will need to implement the following:
 - Use [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) to perform one-hot encoding on the `'features'` data.
 - Convert the target label `'income'` to numerical entries.
   - Set records with "<=50K" to `0` and records with ">50K" to `1`.

In [ ]:
# TODO: One-hot encode the 'features' data using pandas.get_dummies()
features_encoded = None

# TODO: Encode the 'income' data to numerical values
income_encoded = None

# Assign 'features' with the encoded data
features = features_encoded

# Assign 'income' with the encoded data
income = income_encoded

# Print the number of features after one-hot encoding
encoded = list(features.columns)
print "{} total features after one-hot encoding.".format(len(encoded))
#print encoded # Uncomment this line to see the encoded features

### Implementation: Shuffle and Split Data
So far, we have converted all _categorical variables_ into numeric ones. For the next step, we will split the data (both features and their labels) into training and test sets. In the code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`'features'`, `'income'`) into training and testing subsets.
  - Use 80% of the data as a training set, and 20% as testing
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [ ]:
# TODO: Import functionality needed to shuffle and split the data

# TODO: Split the 'feature' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = (None, None, None, None)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

----
## Evaluating Model Performance
WIP DISCUSSION

### Metrics and the Naive Predictor
WIP DISCUSSION

### Question 1 - Naive Predictor Performace
*If we chose a model that always predicted an individual made more than \$50,000, what would that model's accuracy and F1-score be on this dataset?*  
**Note:** You must use the code cell below and assign your results to `'accuracy'` and `'f1'` to be used later.

In [ ]:
# TODO: Calculate accuracy and F1 score on the full dataset
accuracy = None
f1 = None

# Print the results 
print "Naive Predictor: [Accuracy score - {:.4f}, F1 score - {:.4f}]".format(accuracy, f1)

###  Supverised Learning Models
**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent Classifier (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
List three of the supervised learning models above that are appropriate for this problem that you will test on the census data. For each model chosen
- *Describe one real-world application in industry where the model can be applied.* (You may need to do research for this — give references!)
- *What are the strengths of the model; when does it perform well?*
- *What are the weaknesses of the model; when does it perform poorly?*
- *What makes this model a good candidate for the problem, given what you know about the data?*

**Answer: **

### Implementation - Creating a Training and Predicting Pipeline
In the code block below, you will need to implement the following:
 - Import `f1_score` and `accuracy_score` from [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics).
 - Capture the time it takes for the learner to fit the sampled training data.
   - Use two instances of `time()` for this.
 - Capture the time it takes for the learner to predict the testing data.
   - Again, use two instances of `time()` for this.
 - Store the results of the accuracy score from predictions.
 - Store the results of the F1 score from predictions.

In [ ]:
# TODO: Import two metrics from sklearn - f1_score and accuracy_score
from time import time
from sklearn.metrics import f1_score, accuracy_score

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    
    # Start the function
    print "Building a {} model using {} samples.".format(learner.__class__.__name__, sample_size)
    results = {}
    
    # TODO: Get the time using time(), fit the learner to the training data
    #       For the sample_size, get the time again, then store the difference
    start = None
    learner = None
    end = None
    results['train_time'] = None
        
    # TODO: Get the time using time(), predict on the testing data
    #       get the time again, then store the difference
    start = None
    predictions = None
    end = None
    results['pred_time'] = None
        
    # TODO: Compute accuracy on test set
    results['accuracy'] = None
        
    # TODO: Compute F1 on the test set
    results['f1'] = None
       
    # Return the results
    return results

### Implementation: Initial Model Evaluation
WIP DISCUSSION

In the code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `'clf_A'`, `'clf_B'`, and `'clf_C'`.
  - Use a `'random_state'` for each model you use, if provided.
  - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Calculate the number of records equal to 1%, 10%, and 100% of the training data and store those values in `'samples'`
- Run `train_predict()` for each model and store the results in `'results_A'`, `'results_B'`, and `'results_C'`

In [ ]:
# TODO: Import the three supervised learning models from sklearn

# TODO: Initialize the three models
clf_A = None
clf_B = None
clf_C = None

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_1 = None
samples_10 = None
samples_100 = None

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
vs.evaluate(results, accuracy, f1)

----
## Improving Results
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to "Charity-ML" what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

### Question 4 - Describing the Model in Layman's Terms
*In one to two paragraphs, explain to "Charity-ML" in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Initialize the classifier you've chosen and store it in `clf`.
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Perform grid search on the classifier `clf` using `'f1'` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [ ]:
# TODO: Import 'GridSearchCV', 'make_scorer', and any other necessary libraries

# TODO: Initialize the classifier
clf = None

# TODO: Create the parameters list you wish to tune
parameters = None

# TODO: Perform grid search on the classifier using 'f1' as the scoring method
grid_obj = None

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = None

# Get the estimator
best_clf = grid_obj.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print "Unoptimized model\n------"
print "Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions))
print "F1-score on testing data: {:.4f}".format(f1_score(y_test, predictions))
print "\nOptimized Model\n------"
print "Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions))
print "Final F1-score on the testing data: {:.4f}".format(f1_score(y_test, best_predictions))

### Question 5 - Final Model Evaluation
_What is your optimized model's accuracy and F<sub>1</sub> scores on the testing data? Are these scores better or worse than the unoptimized model? How do the results from your optimized model compare to the naive predictor benchmarks you found earlier in **Question 1**?_  
**Hint:** Fill in the table below with your results, and then provide discussion in the **Answer** box.

#### Results:

|     Metric     | Benchmark Predictor | Unoptimized Model | Optimized Model |
| :------------: | :-----------------: | :---------------: | :-------------: | 
| Accuracy Score |               |             |           |
| F1 Score       |                |             |    EXAMPLE      |


**Answer: **

----
## Optional - Feature Importance

An important task when performing supervised learning on a dataset like the census data we study here is determining which features provide the most predictive power. By focusing on the relationship between only a few crucial features and the target label we simplify our understanding of the phenomenon, which is most always a useful thing to do. In the case of this project, that means we wish to identify a small number of features that most strongly predict whether an individual makes at most or more than \$50,000.

Choose a scikit-learn classifier (e.g., adaboost, random forests) that has a `feature_importance_` attribute, which is a function that ranks the importance of features according to the chosen classifier.  In the next python cell fit this classifier to training set and use this attribute to determine the top 5 most important features for the census dataset.

### Question 6 - Feature Relevance Observation
When **Exploring the Data**, it was shown there are thirteen available features for each individual on record in the census data.
_Of these thirteen records, which five features do you believe to be most important for prediction, and in what order would you rank them?_

**Answer:**

### Implementation - Extracting Feature Importances
Choose a `scikit-learn` supervised learning algorithm that has a `feature_importance_` attribute availble for it. This attribute is a function that ranks the importance of each feature when making predictions based on the chosen algorithm.

In the code cell below, you will need to implement the following:
 - Import a supervised learning model from sklearn if it is different from the three used earlier.
 - Train the supervised model on the entire training set.
 - Extract the feature importances using `'.feature_importances_'`.

In [ ]:
# TODO: Import a supervised learning model that has 'feature_importances_' (if needed)

# TODO: Train the supervised model on the training set 
model = None
model = None

# TODO: Extract the feature importances and sort the indices so most relevant is first
importances = None

# Plot
vs.feature_plot(importances, X_train, y_train)

### Question 7 - Feature Importance Analyzed

Observe the visualization created above which displays the five most relevant features for predicting if an individual makes at most or above \$50,000.  
_How do these five features compare to the five features you discussed in **Question 6**? If you were close to the same answer, how does this visualization confirm your thoughts? If you were not close, why do you think these features are more relevant?_

**Answer:**